In [6]:
#arcpy to process LIDAR data    
%load_ext autoreload
%autoreload 2

from arcpy import env
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcpy.sa import *

import sys
sys.path.append("..")

from src.features.create_rasters import *
from src.features.arcpy_scripts import *
from src.features.custom_functions import * 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#define variables - not sure if this is necessary? probably not?

#project geodatabase
#env.workspace = r'C:\Users\rbowers\Desktop\cool_roof_analysis_offnetwork.gdb'
#env.overwriteOutput = True

#dataset file path
#las_dataset = r'I:\Imagery\MassGIS_LAS_files\MMC_lasdataset.lasd'

#arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("NAD 1983 StatePlane Massachusetts FIPS 2001 (Meters)")


### Run these only if need be 

- las_dataset was last created for all MMC on 5/1/2024
- heat_score_mmc was last created for all MMC on 5/1/2024

In [ ]:
#only run this if need be [last run on all MMC: 5/1/2024]
from src.data.make_dataset import heat_fp, las_folder


#las_dataset = create_las_dataset(las_folder=las_folder, las_dataset=las_dataset)

heat_score_mmc = get_heat_score_mmc(heat_index_fp=heat_fp)

### First create slope and intensity rasters for each municipality

Result in geodatabase should be just a slope and intensity raster for each muni 

In [ ]:
mmc_town_names = ['Arlington', 'Boston', 'Braintree', 'Brookline', 'Cambridge', 'Chelsea', 
             'Everett', 'Malden', 'Medford', 'Melrose', 'Newton', 'Quincy', 'Revere', 
             'Somerville', 'Watertown', 'Winthrop']


#note that Revere was removed from this list since it was the test one 

from datetime import datetime

for town_name in mmc_town_names:
    
    print(town_name + ' processing starting at ' + str(datetime.now()))


    las_dataset = create_las_dataset(town_name = town_name, 
                                     las_folder=las_folder)
    

    ndsm_raster = create_ndsm_raster(town_name=town_name,
                                    las_dataset=las_dataset)
    

    slope_raster = create_slope_raster(town_name=town_name,
                                        ndsm_raster=ndsm_raster)
    
    #removing aspect for now, may include later to refine the flat roof analysis
    #aspect_raster = create_aspect_raster(town_name=town_name,
    #                                     ndsm_raster=ndsm_raster)


    intensity_raster = create_intensity_raster(town_name=town_name, 
                                                las_dataset=las_dataset)


### Then create enriched roofprint polygon datasets

In [2]:
town_name = 'Revere'
rooftops_layer = make_coolroof_roofprints_layer(town_name=town_name)

get town boundary
town footprints
extract by mask
reclassify
joining
intensity
zonal stats


In [10]:
from src.data.make_dataset import mmc_heat_blocks
from datetime import datetime


mmc_town_names = ['Arlington', 'Boston', 'Braintree', 'Brookline', 'Cambridge', 'Chelsea', 
             'Everett', 'Malden', 'Medford', 'Melrose', 'Newton', 'Quincy', 'Revere', 
             'Somerville', 'Watertown', 'Winthrop']

for town_name in mmc_town_names:

    print(town_name + ' processing starting at ' + str(datetime.now()))

    cool_roof_footprints_layer = os.path.join(cool_roofs_gdb, (town_name + '_footprints'))

    if arcpy.Exists(cool_roof_footprints_layer): #an existing out_slope_raster would mean this town has already been processed
        
        pass 

    else:    
    
        rooftops_layer = make_coolroof_roofprints_layer(town_name=town_name)
        
        #input for the cool_roof_process() function has to be a gdf
        rooftops_gdf = gpd.read_file(cool_roofs_gdb, layer=(town_name + '_footprints'))

        town_structures_w_info = cool_roof_process(town_name=town_name, 
                                                    rooftops_layer=rooftops_gdf,
                                                    heat_blocks_layer=mmc_heat_blocks)


Arlington processing starting at 2024-05-09 08:52:45.078335
Boston processing starting at 2024-05-09 08:52:45.326340
get town boundary
town footprints
extract by mask
reclassify
joining
intensity
extract by mask, intensity


RuntimeError: The pixel block exceeds the maximum size allowed.

In [ ]:
town_intensity_raster = ExtractByMask(intensity_raster, 
                                      town_boundary,
                                    "INSIDE")

print('reclassify, intensity')

intensity_reclassified = arcpy.management.ReclassifyField(in_table=town_intensity_raster, 
                                                            field='VALUE', 
                                                            method='QUANTILE', 
                                                            classes=10)

## Roof Extraction

Drawing from [this tutorial](https://doc.arcgis.com/en/imagery/workflows/best-practices/las-dataset-to-create-dtm-and-dsm-rasters.htm).
- Also [this video](https://www.youtube.com/watch?v=2pThNwhJ418&t=727s)

Steps:
- Generate a Digital Terrain Model (DTM) (LAS Dataset to Raster)
- Generate a Digital Surface Model (DSM) (LAS Dataset to Raster)
- Generate the Normalized Digital Surface Model (nDSM) (raster calculator)
- Aspect and Slope of nDSM


In [ ]:
## CREATE LAS DATASET ##

point_selection_method = 'CLOSEST_TO_CENTER'

arcpy.CreateLasDataset_management(input=las_folder,
                                  out_las_dataset=las_dataset,
                                  compute_stats='COMPUTE_STATS')
                                  

# Zonal Stats with slope and aspect

In [ ]:
## NDSM ##

In [ ]:
arcpy.management.BuildLasDatasetPyramid(in_las_dataset=las_dataset,
                                 point_selection_method=point_selection_method)